# 👥 Part 3: Building Agent Teams with Agent-as-a-Tool
## Content Creation Studio Workshop

Welcome back to the Google ADK Playbook! In our journey so far, we've built a basic agent
in Part 1 and learned how to create specialist agents with custom tools in Part 2.

This left us with a new challenge: **we now have two separate, expert agents, but they can't
work together**. How do we manage a team of agents?

Today, we'll solve that by building an **Orchestrator**. We'll learn about the powerful
**Agent-as-a-Tool** pattern, where one agent can delegate tasks to other agents, creating
a true multi-agent system.

---

## 📚 The Content Creation Studio Playbook Series
- Part 1: Building Your First AI Agent ✅
- Part 2: Giving Your Agent Custom Tools ✅
- **Part 3: Building Agent Teams with Agent-as-a-Tool** (You are here) 🎯
- Part 4: Multi-Step Workflows with SequentialAgent
- Part 5: Building Iterative Workflows with LoopAgent
- Part 6: Efficient Workflows with ParallelAgent
- Part 7: The Capstone Project

---

## 🎓 New Concepts in This Part

In Part 3, we'll introduce these **new ADK concepts**:

1. **AgentTool Wrapper** - Converting agents into callable tools for other agents
2. **The Orchestrator Pattern** - A coordinator agent that delegates but doesn't execute
3. **Multi-Agent Coordination** - Building teams of agents that work together

Each concept will be clearly marked with 🆕 when first introduced!

---

## 💡 The Challenge: Isolated Specialists

After Part 2, we have:
- `topic_research_agent` - Finds trending topics (uses Google Search)
- `content_analyzer_agent` - Analyzes content quality (uses custom tools)

But they're **isolated**. What if a user asks: *"Research AI productivity topics and analyze their potential"*?

We need a way for agents to work together!

📖 **Documentation**: [Multi-Agent Systems](https://google.github.io/adk-docs/agents/)

## ⚙️ 1. Setup: Install Libraries

First, let's install the Google Agent Development Kit (ADK).

In [ ]:
!pip install google-adk==1.19.0 -q

## 🔑 2. Authentication: Configure Your API Key

Securely provide your Google API key.

In [ ]:
import os
from getpass import getpass

api_key = getpass('Enter your Google API Key: ')
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

## 🛠️ 3. Define Custom Tools

We'll define all the custom tools needed for our specialist agents:
- Content analysis tools (from Part 2)
- New SEO tools for our third specialist

In [ ]:
import re
from typing import List

# --- Content Analysis Tools (from Part 2) ---

def count_words(text: str) -> int:
    """Counts the number of words in the provided text."""
    print(f"🔧 Tool executed: Counting words...")
    words = text.split()
    count = len(words)
    print(f"   Found {count} words")
    return count

def count_syllables(word: str) -> int:
    """Helper function to estimate syllables in a word"""
    word = word.lower()
    vowels = "aeiouy"
    syllable_count = 0
    previous_was_vowel = False
    
    for char in word:
        is_vowel = char in vowels
        if is_vowel and not previous_was_vowel:
            syllable_count += 1
        previous_was_vowel = is_vowel
    
    if word.endswith('e'):
        syllable_count -= 1
    
    return max(1, syllable_count)

def calculate_readability_score(text: str) -> dict:
    """
    Calculates a readability score for the text based on sentence and word complexity.
    Returns a score from 0-100 where higher is easier to read.
    """
    print(f"🔧 Tool executed: Calculating readability score...")
    
    sentences = text.split('.')
    sentences = [s.strip() for s in sentences if s.strip()]
    
    if not sentences:
        return {"score": 0, "grade": "Unable to calculate"}
    
    words = text.split()
    total_words = len(words)
    total_sentences = len(sentences)
    total_syllables = sum(count_syllables(word) for word in words)
    
    if total_words == 0 or total_sentences == 0:
        score = 0
    else:
        score = 206.835 - 1.015 * (total_words / total_sentences) - 84.6 * (total_syllables / total_words)
        score = max(0, min(100, score))
    
    if score >= 60:
        grade = "Easy to read"
    elif score >= 50:
        grade = "Moderate"
    else:
        grade = "Complex"
    
    result = {"score": round(score, 2), "grade": grade}
    print(f"   Score: {result['score']} - {result['grade']}")
    return result

def generate_hashtags(text: str, count: int = 5) -> List[str]:
    """Generates relevant hashtags from text by extracting key terms."""
    print(f"🔧 Tool executed: Generating {count} hashtags...")
    
    stop_words = {
        'the', 'is', 'at', 'which', 'on', 'a', 'an', 'as', 'are', 'was', 'were',
        'been', 'be', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would',
        'could', 'should', 'may', 'might', 'must', 'can', 'of', 'to', 'for', 'in',
        'with', 'by', 'from', 'up', 'about', 'into', 'through', 'and', 'or', 'but'
    }
    
    words = re.findall(r'\b[a-zA-Z]{4,}\b', text.lower())
    word_freq = {}
    for word in words:
        if word not in stop_words:
            word_freq[word] = word_freq.get(word, 0) + 1
    
    sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    top_words = [word for word, freq in sorted_words[:count]]
    hashtags = [f"#{word.capitalize()}" for word in top_words]
    
    print(f"   Generated: {', '.join(hashtags)}")
    return hashtags

# --- NEW: SEO Tools ---

def extract_keywords(text: str, count: int = 10) -> List[str]:
    """Extracts the most important keywords from text for SEO purposes."""
    print(f"🔧 Tool executed: Extracting {count} keywords...")
    
    stop_words = {
        'the', 'is', 'at', 'which', 'on', 'a', 'an', 'as', 'are', 'was', 'were',
        'been', 'be', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would',
        'their', 'they', 'what', 'when', 'where', 'who', 'why', 'how'
    }
    
    words = re.findall(r'\b[a-zA-Z]{3,}\b', text.lower())
    word_freq = {}
    for word in words:
        if word not in stop_words:
            word_freq[word] = word_freq.get(word, 0) + 1
    
    sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    keywords = [word for word, freq in sorted_words[:count]]
    
    print(f"   Keywords: {', '.join(keywords)}")
    return keywords

def calculate_seo_score(text: str, target_keywords: List[str]) -> dict:
    """Calculates an SEO score based on keyword density, content length, and structure."""
    print(f"🔧 Tool executed: Calculating SEO score...")
    
    text_lower = text.lower()
    words = text.split()
    word_count = len(words)
    
    keywords_found = sum(1 for keyword in target_keywords if keyword.lower() in text_lower)
    keyword_coverage = (keywords_found / len(target_keywords) * 100) if target_keywords else 0
    
    total_keyword_mentions = sum(text_lower.count(keyword.lower()) for keyword in target_keywords)
    keyword_density = (total_keyword_mentions / word_count * 100) if word_count > 0 else 0
    
    if word_count < 300:
        length_score = 20
    elif word_count < 800:
        length_score = 50
    elif word_count <= 2000:
        length_score = 100
    else:
        length_score = 80
    
    seo_score = (keyword_coverage * 0.5 + length_score * 0.5)
    
    result = {
        "seo_score": round(seo_score, 2),
        "keyword_coverage": f"{round(keyword_coverage, 2)}%",
        "keyword_density": f"{round(keyword_density, 2)}%",
        "word_count": word_count
    }
    
    print(f"   SEO Score: {result['seo_score']}/100")
    return result

print("✅ All custom tools defined!")

## 🤖 4. Create Specialist Agents

Now we'll create **three specialist agents**, each an expert in their domain.

**Recall from Part 2**: The Specialist Agent Pattern means one agent focuses on one domain with related tools.

### Agent 1: Topic Research Specialist
- **Tool**: Google Search
- **Expertise**: Finding trending blog topics

### Agent 2: Content Analysis Specialist
- **Tools**: `count_words`, `calculate_readability_score`, `generate_hashtags`
- **Expertise**: Analyzing content quality

### Agent 3: SEO Optimization Specialist
- **Tools**: `extract_keywords`, `calculate_seo_score`
- **Expertise**: SEO analysis and optimization

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search

# Agent 1: Topic Research Specialist
topic_research_agent = Agent(
    name="topic_research_agent",
    model="gemini-2.5-flash",
    description="A specialist agent that researches trending blog topics using search.",
    instruction="""
    You are an expert content strategist and topic researcher. Your only goal is to find
    compelling blog topics using your search tool.
    
    Your process:
    1. Use search to find trending topics in the specified niche
    2. Suggest 5-7 specific blog post titles
    3. Explain why each topic is valuable
    
    You do not analyze content or optimize SEO. Stick to topic research.
    """,
    tools=[google_search]
)

# Agent 2: Content Analysis Specialist
content_analyzer_agent = Agent(
    name="content_analyzer_agent",
    model="gemini-2.5-flash",
    description="A specialist agent for analyzing blog content quality and readability.",
    instruction="""
    You are a content analysis expert. Your only job is to analyze text quality.
    
    When given content, use your tools to:
    1. Count words
    2. Calculate readability score
    3. Generate hashtags for social promotion
    
    Provide clear feedback on content quality. You do not research topics or optimize SEO.
    """,
    tools=[count_words, calculate_readability_score, generate_hashtags]
)

# Agent 3: SEO Optimization Specialist
seo_optimizer_agent = Agent(
    name="seo_optimizer_agent",
    model="gemini-2.5-flash",
    description="A specialist agent for SEO analysis and keyword optimization.",
    instruction="""
    You are an SEO optimization expert. Your only job is to analyze and improve SEO.
    
    Your process:
    1. Extract keywords from content using `extract_keywords`
    2. Calculate SEO score using `calculate_seo_score` with the extracted keywords
    3. Provide specific, actionable SEO recommendations
    
    You do not research topics or analyze general content quality. Stick to SEO.
    """,
    tools=[extract_keywords, calculate_seo_score]
)

print("🧞 All specialist agents created!")

## 🔧 5. The Agent-as-a-Tool Pattern

### 🆕 NEW CONCEPT: AgentTool Wrapper

> **What is AgentTool?**  
> AgentTool is a wrapper class that converts an entire Agent into a callable tool for other agents. This enables **agent-to-agent delegation** and is the foundation of multi-agent systems in ADK.
>
> 📖 **Reference**: [Multi-Agent Systems](https://google.github.io/adk-docs/agents/)

### The Problem:
Our three specialists are isolated. They can't call each other.

### The Solution: Agent-as-a-Tool Pattern
The ADK provides the **`AgentTool`** class that wraps an agent, making it callable as a tool by another agent!

### How It Works:
```python
# Take an agent...
my_agent = Agent(name="specialist", ...)

# ...wrap it as a tool...
my_tool = AgentTool(agent=my_agent)

# ...give it to another agent!
orchestrator = Agent(name="boss", tools=[my_tool], ...)
```

The orchestrator can now **delegate** tasks to the specialist by calling the tool!

### Key Benefits:
- **Composability**: Agents become reusable building blocks
- **Separation of Concerns**: Each agent maintains its own expertise
- **Scalability**: Easy to add new specialists to the team

In [ ]:
from google.adk.tools.agent_tool import AgentTool

# Wrap each specialist agent as a tool
topic_research_tool = AgentTool(agent=topic_research_agent)
content_analyzer_tool = AgentTool(agent=content_analyzer_agent)
seo_optimizer_tool = AgentTool(agent=seo_optimizer_agent)

print("✅ All agents wrapped as tools!")

## 👔 6. Create the Orchestrator Agent

### 🆕 NEW CONCEPT: The Orchestrator Pattern

> **What is an Orchestrator?**  
> An orchestrator is a coordinator agent whose sole responsibility is to **understand user intent and delegate to appropriate specialists**. It doesn't execute tasks itself—it routes work to the right experts.
>
> **Key Characteristics**:
> - No special skills of its own
> - Understands user requests and routing logic
> - Uses AgentTools to delegate to specialists
> - Presents specialist responses to users
>
> 📖 **Reference**: [Multi-Agent Systems](https://google.github.io/adk-docs/agents/)

With our specialists now available as tools, we can build our **manager agent**.

### How It Decides:
Its instructions explicitly tell it:
- Topic research questions → use `topic_research_tool`
- Content analysis questions → use `content_analyzer_tool`
- SEO questions → use `seo_optimizer_tool`

### Architecture:
```
User Query → Orchestrator → Analyzes Intent → Delegates to Specialist → Returns Result
```

In [ ]:
content_orchestrator_agent = Agent(
    name="content_orchestrator_agent",
    model="gemini-2.5-flash",
    description="The main orchestrator that delegates content tasks to specialist agents.",
    instruction="""
    You are the root orchestrator managing content creation tasks. Your job is to
    understand the user's request and delegate it to the correct specialist tool.
    
    - If the user wants to research blog topics, find trending content ideas, or
      discover what to write about, use the `topic_research_tool`.
    
    - If the user wants to analyze existing content, check readability, count words,
      or generate hashtags, use the `content_analyzer_tool`.
    
    - If the user wants SEO analysis, keyword extraction, or SEO optimization,
      use the `seo_optimizer_tool`.
    
    Do not try to answer the user's query directly. Always delegate to the appropriate
    tool. Present the specialist's response clearly to the user.
    """,
    tools=[topic_research_tool, content_analyzer_tool, seo_optimizer_tool]  # 🔧 AgentTools!
)

print("🧞 Orchestrator agent created!")

## 🚀 7. Build the Execution Engine

Our standard helper function for running queries with proper session management.

In [ ]:
from IPython.display import display, Markdown
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

# Initialize Session Service
session_service = InMemorySessionService()
user_id = "adk_content_creator_001"

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response

print("✅ Execution engine ready!")

## ✨ 8. Test the Multi-Agent System!

### 🆕 NEW CONCEPT: Multi-Agent Coordination

> **What is Multi-Agent Coordination?**  
> Multi-agent coordination is the pattern where multiple specialized agents work together, coordinated by an orchestrator, to solve complex problems. Each agent maintains its own expertise while contributing to a larger system.
>
> **Key Elements**:
> - **Specialists**: Focused agents with specific capabilities
> - **Orchestrator**: Routes requests to appropriate specialists
> - **Communication**: Via tool calls and responses
> - **State Management**: Each agent has its own session context
>
> 📖 **Reference**: [Multi-Agent Systems](https://google.github.io/adk-docs/agents/)

Now let's test our orchestrator with different types of queries. Watch how it intelligently
routes each query to the appropriate specialist!

### Query 1: Topic Research
The orchestrator will route this to `topic_research_tool`

### Query 2: Content Analysis
The orchestrator will route this to `content_analyzer_tool`

### Query 3: SEO Analysis
The orchestrator will route this to `seo_optimizer_tool`

In [ ]:
async def run_stateful_orchestrator():
    """Run the orchestrator with multiple queries in the same session"""
    session = await session_service.create_session(
        app_name=content_orchestrator_agent.name,
        user_id=user_id
    )

    # Query 1: Topic Research
    query1 = "I need blog topic ideas for a sustainable living blog focused on zero-waste lifestyle"
    print(f"👤 User: {query1}\n")
    await run_agent_query(content_orchestrator_agent, query1, session, user_id)

    # Query 2: Content Analysis
    sample_content = """
    Living a zero-waste lifestyle doesn't have to be overwhelming. Start small by making
    simple swaps in your daily routine. Replace plastic bags with reusable cloth bags.
    Choose a refillable water bottle instead of buying disposable ones. These small changes
    add up over time and make a significant environmental impact.
    """
    query2 = f"Can you analyze this blog post draft:\n\n{sample_content}"
    print(f"👤 User: {query2}\n")
    await run_agent_query(content_orchestrator_agent, query2, session, user_id)

    # Query 3: SEO Analysis
    query3 = "Now analyze the SEO quality of that same content"
    print(f"👤 User: {query3}\n")
    await run_agent_query(content_orchestrator_agent, query3, session, user_id)

# Run the full system
await run_stateful_orchestrator()

## 🎉 Recap: What We've Learned

This was a major leap forward in building sophisticated AI systems:

### Core Concepts Introduced:

1. **🆕 AgentTool Wrapper** - Converts agents into callable tools for other agents  
   📖 [Multi-Agent Systems](https://google.github.io/adk-docs/agents/)

2. **🆕 The Orchestrator Pattern** - Coordinator agent that delegates but doesn't execute  
   Pattern: Understand intent → Route to specialist → Present result

3. **🆕 Multi-Agent Coordination** - Teams of specialized agents working together  
   Architecture: User → Orchestrator → Specialists → Results

### Key Takeaways:

- **Specialization**: Break down complex problems into focused agents
- **Composition**: Agents become reusable building blocks via AgentTool
- **Delegation**: Orchestrator understands intent and routes appropriately
- **Stateful Conversations**: Session maintains context across queries
- **Separation of Concerns**: Each agent focuses on its expertise

### Architecture Pattern:
```
User → Orchestrator → (delegates) → Specialist Agents
                                    - Topic Research
                                    - Content Analyzer  
                                    - SEO Optimizer
```

---

## 🚀 What's Next?

We've successfully built an agent team managed by an orchestrator! But what if one of
the tasks requires multiple, **ordered steps**? For example, how do we ensure the agent
first researches a topic and then writes content based on that research?

In **Part 4**, we'll learn how to build these kinds of ordered, multi-step workflows
using the **SequentialAgent** - a workflow agent that executes sub-agents in sequence.

### Preview of Part 4 Concepts:
- 🆕 SequentialAgent (Workflow Agent)
- 🆕 output_key Parameter
- 🆕 State Passing Between Agents
- 🆕 Variable Interpolation

See you in Part 4! 🚀